In [1]:
# Instalamos pyspark y py4j
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7fd6710321b0eeb7689dc2835151c77c7479fd338827841587d2d3c0b0e6a216
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

Spark = SparkSession.builder.appName("jjta").getOrCreate()
sc = Spark.sparkContext

### EJERCICIO #1  Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python,
R, C, Scala, Rugby y SQL.






In [7]:
#CREACION DE RDD
rdd_lenguajes = sc.parallelize(["Python", "R", "C", "Scala", "Rugby", "SQL"])
rdd_lenguajes.collect()

['Python', 'R', 'C', 'Scala', 'Rugby', 'SQL']

      A. Cree un nuevo RDD que solo contenga aquellos lenguajes de
          programación que comiencen con la letra R.

---



In [40]:
#FILTRAMOS EL RDD PARA QUE NOS MUESTRE LOS LENGUAJES DE PROGRAMACION QUE INICIAN CON "R"
rdd_lenguajes_con_r = rdd_lenguajes.filter(lambda x: x.startswith('R'))
rdd_lenguajes_con_r.collect()

['R', 'Rugby']

### EJERCICIO #2 Cree un RDD llamado pares que contenga los números pares existentes en el intervalo "20;**30**".




In [15]:
#CREACION Y FILTRADO DE RDD
lista = sc.parallelize(list(range(20, 31)))
rdd_pares = lista.filter(lambda x: x% 2 == 0)
rdd_pares.collect()

[20, 22, 24, 26, 28, 30]

      B. Obtenga una lista compuesta por los números pares en el intervalo
         "20;30" y sus respectivas raíces cuadradas. Un ejemplo del resultado
         deseado para el intervalo "50;60" sería la lista "50, 7.0710678118654755, 52, 7.211102550927978,
         54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834".

In [18]:
#CREACION DE LISTA COMPUESTA USANDO MAP
rdd_sqrt = rdd_pares.map(lambda x: (x, x ** 0.5))
rdd_sqrt.collect()

[(20, 4.47213595499958),
 (22, 4.69041575982343),
 (24, 4.898979485566356),
 (26, 5.0990195135927845),
 (28, 5.291502622129181),
 (30, 5.477225575051661)]

      C. Eleve el número de particiones del RDD sqrt a 20.

In [20]:
#INICIALMENTE POR DEFECTO TENIAMOS 2 PARTICIONES
lista = sc.parallelize(list(range(20, 31)))

lista.getNumPartitions()

2

In [21]:
#AUMENTAMOS LA PARTICION AGRGANDO EL ",20"
lista = sc.parallelize(list(range(20, 31)), 20)

lista.getNumPartitions()

20

In [24]:
#USAMOS LA FUNCION REPARTITION PARA AUMENTAR LAS PARTICIONES
lista = lista.repartition(20)
lista.getNumPartitions()

20

      D. Si tuviera que disminuir el número de particiones luego de haberlo
          establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?

      PARA DISMINUIR EL NUMERO DE PARTICIONES USO "coalesce()",
      esta funcion me permite reducir el numero de particiones
      en este ejemplo lo redusco de 20 a 4

In [25]:
#USAMOS COALESCE PARA DISMINUIR LAS PARTICIONES
lista = lista.coalesce(4)
lista.getNumPartitions()

4

### EJERCICIO #3 Cree un RDD del tipo clave valor a partir de la lectura del txt de transacciones.
Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado. Supongamos
que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de
cuentas. Obtenga el monto total por cada cuenta (Suma).
Tip: Cree su propia función para procesar el RDD leído.

In [32]:
#LEEMOS EL TEXTO
rdd_transacciones = sc.textFile('./transacciones')
rdd_transacciones.collect()

['(1001, 52.3)',
 '(1005, 20.8)',
 '(1001, 10.1)',
 '(1004, 52.7)',
 '(1005, 20.7)',
 '(1002, 85.3)',
 '(1004, 20.9)']

In [38]:
#CONVERTIMOS EL CONTENIDO A CLAVE VALOR
rdd_transacciones_clave_valor = sc.textFile('./transacciones').map(lambda x: tuple(map(float, x.strip('()').split(','))))
rdd_transacciones_clave_valor.collect()

[(1001.0, 52.3),
 (1005.0, 20.8),
 (1001.0, 10.1),
 (1004.0, 52.7),
 (1005.0, 20.7),
 (1002.0, 85.3),
 (1004.0, 20.9)]

In [39]:
#MONTO TOTAL POR CADA CUENTA (SUMA)
rdd_reduccion = rdd_transacciones_clave_valor.reduceByKey(lambda x,y: x + y)
rdd_reduccion.collect()

[(1004.0, 73.6), (1002.0, 85.3), (1001.0, 62.4), (1005.0, 41.5)]